<a href="https://colab.research.google.com/github/jingjingchen149/new-plan/blob/main/vits_jap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/innnky/vits-japanese.git

Cloning into 'vits-japanese'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 96 (delta 15), reused 21 (delta 9), pack-reused 41
Unpacking objects: 100% (96/96), done.


In [ ]:
!pip install pypinyin Cython==0.29.21 librosa==0.8.0 matplotlib==3.3.1 numpy==1.18.5 phonemizer==2.2.1 scipy==1.5.2 Unidecode==1.1.1 >log.log
!sudo apt-get install espeak -y >log.log
!pip install gdown >log.log 
!pip install pyopenjtalk janome > log.log
!pip install cloud-tpu-client > log.log

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.18.5 which is incompatible.
tensorflow 2.8.2+zzzcolab20220719082949 requires numpy>=1.20, but you have numpy 1.18.5 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.18.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.5.2 which is incompatible.
jaxlib 0.3.14+cuda11.cudnn805 requires numpy>=1.19, but you have numpy 1.18.5 which is incompatible.
jax 0.3.14 requires numpy>=1.19, but you have numpy 1.18.5 which is incompatible.
cmdstanpy 1.0.4 requires numpy>=1.21, but you have numpy 1.18.5 which is incompatible.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/

In [ ]:
%cd /content/vits-japanese/monotonic_align
!python setup.py build_ext --inplace 


/content/vits-japanese/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/vits-japanese/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'monotonic_align.core' extension
creating build
creating build/temp.linux-x86_64-3.7
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I/usr/include/python3.7m -c core.c -o build/temp.linux-x86_64-3.7/core.o
creating /content/vits-japanese/monotonic_align/monotonic_align
x86_64-linux-gnu-gcc -pt

In [ ]:
%cd /content/vits-japanese

import logging

numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

import librosa

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text.cleaners import japanese_phrase_cleaners
from text import cleaned_text_to_sequence
from pypinyin import lazy_pinyin, Style

from scipy.io.wavfile import write

def get_text(text, hps):
    text_norm = cleaned_text_to_sequence(text)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm
# hps_ms = utils.get_hparams_from_file("./configs/vctk_base.json")
hps = utils.get_hparams_from_file("./configs/nan.json")
# net_g_ms = SynthesizerTrn(
#     len(symbols),
#     hps_ms.data.filter_length // 2 + 1,
#     hps_ms.train.segment_size // hps.data.hop_length,
#     n_speakers=hps_ms.data.n_speakers,
#     **hps_ms.model)

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

def tts(text):
  stn_tst = get_text(text, hps)
  with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.float().numpy()
  ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))
def jtts(text):
  stn_tst = get_text(japanese_phrase_cleaners(text), hps)
  with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.float().numpy()
  ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))


/content/vits-japanese


In [ ]:
# 视频发布时模型
# !gdown "https://drive.google.com/u/1/uc?id=1pGYpv2ZIv9oN4sfFF0zZLs61i-rwMUGK&export=download"
# 最新预训练模型
!gdown "https://drive.google.com/u/0/uc?id=169Hbf3Qs4ahQ8EFRZCQtlGtVvp2f1Dd0&export=download" 

Downloading...
From: https://drive.google.com/u/0/uc?id=169Hbf3Qs4ahQ8EFRZCQtlGtVvp2f1Dd0&export=download
To: /content/vits-japanese/G_84000.pth
100% 436M/436M [00:03<00:00, 141MB/s]


In [ ]:
# _ = utils.load_checkpoint("/content/vits-japanese/G_69000.pth", net_g, None)
# 最新预训练模型
_ = utils.load_checkpoint("/content/vits-japanese/G_84000.pth", net_g, None)


INFO:root:Loaded checkpoint '/content/vits-japanese/G_84000.pth' (iteration 460)


In [ ]:
tts("nanika acltara itsudemo hanashIte kudasai. gakuiNno kotojanaku, shijini kaNsuru kotodemo nanidemo.")
tts("soodesUyo. jitsuwa watashitachiga nyuugakU suru maekara aclta bukatsuna NdesU. imawa buiNga inakUte, shozoku shIteru nowa watashI hitoridesUga.")
tts("nanode saikiNwa, gakUseekaino hookarawa keclkoo tsUtsukaretete...... katsudoomo, haclpyoonadoo iyokutekini shIte iru wakejaarimaseNkarane.")
tts("ichioo. shiclte haimasUkedo, watashini dekiru nowa tarocltoguraidesU. akumade shumiteedodesUkara.")
tts("a, hai. suguni ikimasU.")
tts("toshoshItsuwa...... moo daremo inai NdesUka?")

In [ ]:
jtts("吾輩は猫である。名前はまだない")
jtts("試験勉強頑張ってくださいね")
jtts("私の処女をもらってください")

Downloading: "https://github.com/r9y9/open_jtalk/releases/download/v1.11.1/open_jtalk_dic_utf_8-1.11.tar.gz"


dic.tar.gz: 0.00B [00:00, ?B/s]

Extracting tar file /usr/local/lib/python3.7/dist-packages/pyopenjtalk/dic.tar.gz


In [ ]:
jtts("なんでこんなに慣れてんのよ。私の方が先に好きだったのに")
jtts("大変に気分がいい") 

In [ ]:
jtts("私わあやちねねです")

In [ ]:
jtts("私のおなにいを見ないでください!")

In [ ]:
jtts("それわどうかな")

In [ ]:
jtts("授業中に出したら、学校生活終わるなり")